In [37]:
import os
import time
import math
import datetime

import gc

from multiprocessing import Pool, cpu_count

import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder

from functools import reduce

In [2]:
train = pd.read_csv("round1_ijcai_18_train_20180301.txt", sep=" ")
train.head()

,instance_id,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,...,context_page_id,predict_category_property,shop_id,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description,is_trade
0,108641074714126964,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4006,5799347067982556520:-1;509660095530134768:-1;5...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
1,5754713551599725161,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4001,5799347067982556520:9172976955054793469;790838...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
2,842679481291040981,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4001,5799347067982556520:5131280576272319091;725801...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
3,937088850059189027,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4016,509660095530134768:-1;5799347067982556520:-1;7...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
4,7975697065017708072,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4001,5799347067982556520:9172976955054793469;790838...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0


In [3]:
train.columns

Index(['instance_id', 'item_id', 'item_category_list', 'item_property_list',
       'item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'user_id', 'user_gender_id',
       'user_age_level', 'user_occupation_id', 'user_star_level', 'context_id',
       'context_timestamp', 'context_page_id', 'predict_category_property',
       'shop_id', 'shop_review_num_level', 'shop_review_positive_rate',
       'shop_star_level', 'shop_score_service', 'shop_score_delivery',
       'shop_score_description', 'is_trade'],
      dtype='object')

# data.py

In [4]:
def today(x):
    return time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x))

In [5]:
def getday(x):
    day=int(x.split(' ')[0].split('-')[-1])
    if day==31:
        day=0
    return day

In [6]:
train["day"] = train["context_timestamp"].apply(lambda x: getday(today(x)))

train["day"]

0         18
1         18
2         18
3         18
4         18
          ..
478133    24
478134    24
478135    24
478136    24
478137    24
Name: day, Length: 478138, dtype: int64

In [7]:
train["hour"] = train["context_timestamp"].apply(lambda x:int(today(x).split()[1].split(':')[0]))
train["hour"]

0         10
1         12
2          3
3          6
4         19
          ..
478133     7
478134    23
478135    20
478136    18
478137    19
Name: hour, Length: 478138, dtype: int64

In [8]:
train["context_timestamp"] = train["context_timestamp"].apply(lambda x: today(x))
train["context_timestamp"]

0         2018-09-18 10:09:04
1         2018-09-18 12:00:32
2         2018-09-18 03:04:12
3         2018-09-18 06:17:50
4         2018-09-18 19:48:40
                 ...         
478133    2018-09-24 07:48:39
478134    2018-09-24 23:03:35
478135    2018-09-24 20:45:00
478136    2018-09-24 18:08:49
478137    2018-09-24 19:48:21
Name: context_timestamp, Length: 478138, dtype: object

In [9]:
def gethour(x):
    hour=int(x.split(' ')[1].split(':')[0])
    minute=int(x.split(' ')[1].split(':')[1])
    minute=1 if minute>=30 else 0

    return hour*2+minute

In [10]:
train["hour48"] = train["context_timestamp"].apply(gethour)
train["hour48"]

0         20
1         24
2          6
3         12
4         39
          ..
478133    15
478134    46
478135    41
478136    36
478137    39
Name: hour48, Length: 478138, dtype: int64

In [11]:
def same_cate(x):
    cate = set(x['item_category_list'].split(';'))
    cate2 = set([i.split(':')[0] for i in x['predict_category_property'].split(';')])
    
    return len(cate & cate2)

In [12]:
train["item_category_list"]

0         7908382889764677758;5799347067982556520
1         7908382889764677758;5799347067982556520
2         7908382889764677758;5799347067982556520
3         7908382889764677758;5799347067982556520
4         7908382889764677758;5799347067982556520
                           ...                   
478133    7908382889764677758;5755694407684602296
478134    7908382889764677758;5755694407684602296
478135    7908382889764677758;5755694407684602296
478136    7908382889764677758;5755694407684602296
478137    7908382889764677758;5755694407684602296
Name: item_category_list, Length: 478138, dtype: object

In [13]:
train["predict_category_property"]

0         5799347067982556520:-1;509660095530134768:-1;5...
1         5799347067982556520:9172976955054793469;790838...
2         5799347067982556520:5131280576272319091;725801...
3         509660095530134768:-1;5799347067982556520:-1;7...
4         5799347067982556520:9172976955054793469;790838...
                                ...                        
478133    8257512457089702259:8895425924056041189;227312...
478134    5755694407684602296:-1;7908382889764677758:-1;...
478135        5755694407684602296:-1;7908382889764677758:-1
478136    8257512457089702259:-1;1760164811125093110:203...
478137    5755694407684602296:-1;509660095530134768:-1;8...
Name: predict_category_property, Length: 478138, dtype: object

In [14]:
train["same_cate"] = train.apply(same_cate, axis=1)
train["same_cate"]

0         2
1         2
2         2
3         2
4         2
         ..
478133    2
478134    2
478135    2
478136    2
478137    2
Name: same_cate, Length: 478138, dtype: int64

In [15]:
def same_property(x):
    property_a = set(x['item_property_list'].split(';'))
    
    a = []
    for i in [i.split(':')[1].split(',') for i in x['predict_category_property'].split(';') if len(i.split(':')) > 1]:
        a += i
    
    property_b = set(a)
    
    return len(property_a & property_b)

In [16]:
train["same_property"] = train.apply(same_property, axis=1)
train["same_property"]

0         0
1         1
2         1
3         0
4         1
         ..
478133    2
478134    1
478135    0
478136    0
478137    0
Name: same_property, Length: 478138, dtype: int64

In [17]:
train["property_num"] = train['item_property_list'].apply(lambda x:len(x.split(';')))
train["property_num"]

0         22
1         22
2         22
3         22
4         22
          ..
478133    48
478134    48
478135    48
478136    48
478137    26
Name: property_num, Length: 478138, dtype: int64

In [18]:
train['pred_cate_num'] = train['predict_category_property'].apply(lambda x:len(x.split(';')))
train["pred_cate_num"]

0         5
1         2
2         3
3         5
4         2
         ..
478133    6
478134    4
478135    2
478136    7
478137    4
Name: pred_cate_num, Length: 478138, dtype: int64

In [19]:
def f(x):
    try:
        return len([i for i in reduce((lambda x, y: x + y), [i.split(':')[1].split(',') for i in x.split(';') if len(i.split(':'))>1]) if i != '-1'])
    except:
        return 0

In [20]:
train['pred_prop_num'] = train['predict_category_property'].apply(f)
train["pred_prop_num"]

0          1
1          6
2          3
3         10
4          2
          ..
478133     6
478134    13
478135     0
478136     4
478137     0
Name: pred_prop_num, Length: 478138, dtype: int64

In [21]:
train['query1'] = train['predict_category_property'].apply(lambda x:x.split(';')[0].split(':')[0])
train["query1"]

0         5799347067982556520
1         5799347067982556520
2         5799347067982556520
3          509660095530134768
4         5799347067982556520
                 ...         
478133    8257512457089702259
478134    5755694407684602296
478135    5755694407684602296
478136    8257512457089702259
478137    5755694407684602296
Name: query1, Length: 478138, dtype: object

In [22]:
train['query'] = train['predict_category_property'].apply(lambda x:'-'.join(sorted([i.split(':')[0] for i in [i for i in x.split(';')]])))
train["query"]

0         509660095530134768-5755694407684602296-5799347...
1                   5799347067982556520-7908382889764677758
2         5799347067982556520-7258015885215914736-790838...
3         1950314698730389427-509660095530134768-5799347...
4                   5799347067982556520-7908382889764677758
                                ...                        
478133    22731265849056483-5755694407684602296-79083828...
478134    1923130679917048904-2356297995131360540-575569...
478135              5755694407684602296-7908382889764677758
478136    1760164811125093110-5755694407684602296-790838...
478137    509660095530134768-5755694407684602296-7908382...
Name: query, Length: 478138, dtype: object

In [23]:
train['cate'] = train['item_category_list'].apply(lambda x: x.split(';')[1])
train["cate"]

0         5799347067982556520
1         5799347067982556520
2         5799347067982556520
3         5799347067982556520
4         5799347067982556520
                 ...         
478133    5755694407684602296
478134    5755694407684602296
478135    5755694407684602296
478136    5755694407684602296
478137    5755694407684602296
Name: cate, Length: 478138, dtype: object

In [24]:
def fillna(data):
    numeric_feature = ['day', 'item_price_level', 'item_sales_level', 'item_collected_level', 'item_pv_level',
                       'user_age_level', 'user_star_level', 'shop_review_num_level',
                       'shop_review_positive_rate', 'shop_star_level', 'shop_score_service', 'shop_score_delivery',
                       'shop_score_description', 'context_page_id'
                       ]
    string_feature = ['shop_id', 'item_id', 'user_id', 'item_brand_id', 'item_city_id', 'user_gender_id',
                      'user_occupation_id', 'context_page_id', 'hour']
    other_feature = ['item_property_list', 'predict_category_property']
    
    #填充缺失值
    for i in string_feature+other_feature:
        mode_num = data[i].mode()[0]
        if (mode_num != -1):
            print(i)
            data.loc[data[i] == -1, i] = mode_num
        else:
            print(-1)
    
    for i in numeric_feature:
        mean_num = data[i].mean()
        if (mean_num != -1):
            print(i)
            data.loc[data[i] == -1, i] = mean_num
        else:
            print(-1)
    
    return data

In [25]:
train = fillna(train.copy())

shop_id
item_id
user_id
item_brand_id
item_city_id
user_gender_id
user_occupation_id
context_page_id
hour
item_property_list
predict_category_property
day
item_price_level
item_sales_level
item_collected_level
item_pv_level
user_age_level
user_star_level
shop_review_num_level
shop_review_positive_rate
shop_star_level
shop_score_service
shop_score_delivery
shop_score_description
context_page_id


In [26]:
"""
统计属性出现的次数，取top1的属性作为特征，top1-5合并作为特征
预测的属性，top1,合并top1-5
"""
def property_feature(org):
    tmp=org['item_property_list'].apply(lambda x:x.split(';')).values
    property_dict={}
    property_list=[]
    
    for i in tmp:
        property_list+=i
    
    for i in property_list:
        if i in property_dict:
            property_dict[i] += 1
        else:
            property_dict[i] = 1
    
    print('dict finish')
    
    def top(x):
        properties=x.split(';')
        cnt=[property_dict[i] for i in properties]
        res=sorted(zip(properties,cnt),key=lambda x:x[1],reverse=True)
        top1=res[0][0]
        top2 = '_'.join([i[0] for i in res[:2]])
        top3 = '_'.join([i[0] for i in res[:3]])
        top4 = '_'.join([i[0] for i in res[:4]])
        top5='_'.join([i[0] for i in res[:5]])
        top10 = '_'.join([i[0] for i in res[:10]])
        return (top1,top2,top3,top4,top5,top10)
    
    org['top']=org['item_property_list'].apply(top)
    print('top finish')
    
    org['top1']=org['top'].apply(lambda x:x[0])
    org['top2'] = org['top'].apply(lambda x: x[1])
    org['top3'] = org['top'].apply(lambda x: x[2])
    org['top4'] = org['top'].apply(lambda x: x[3])
    org['top5'] = org['top'].apply(lambda x: x[4])
    org['top10'] = org['top'].apply(lambda x: x[5])
    
    return org[['instance_id','top1','top2','top3','top4','top5','top10']]

In [27]:
property_feature(train)

dict finish
top finish


,instance_id,top1,top2,top3,top4,top5,top10
0,108641074714126964,2636395404473730413,2636395404473730413_5131280576272319091,2636395404473730413_5131280576272319091_178243...,2636395404473730413_5131280576272319091_178243...,2636395404473730413_5131280576272319091_178243...,2636395404473730413_5131280576272319091_178243...
1,5754713551599725161,2636395404473730413,2636395404473730413_5131280576272319091,2636395404473730413_5131280576272319091_178243...,2636395404473730413_5131280576272319091_178243...,2636395404473730413_5131280576272319091_178243...,2636395404473730413_5131280576272319091_178243...
2,842679481291040981,2636395404473730413,2636395404473730413_5131280576272319091,2636395404473730413_5131280576272319091_178243...,2636395404473730413_5131280576272319091_178243...,2636395404473730413_5131280576272319091_178243...,2636395404473730413_5131280576272319091_178243...
3,937088850059189027,2636395404473730413,2636395404473730413_5131280576272319091,2636395404473730413_5131280576272319091_178243...,2636395404473730413_5131280576272319091_178243...,2636395404473730413_5131280576272319091_178243...,2636395404473730413_5131280576272319091_178243...
4,7975697065017708072,2636395404473730413,2636395404473730413_5131280576272319091,2636395404473730413_5131280576272319091_178243...,2636395404473730413_5131280576272319091_178243...,2636395404473730413_5131280576272319091_178243...,2636395404473730413_5131280576272319091_178243...
...,...,...,...,...,...,...,...
478133,5940763769799191887,2636395404473730413,2636395404473730413_5131280576272319091,2636395404473730413_5131280576272319091_124376...,2636395404473730413_5131280576272319091_124376...,2636395404473730413_5131280576272319091_124376...,2636395404473730413_5131280576272319091_124376...
478134,3387284546470665526,2636395404473730413,2636395404473730413_5131280576272319091,2636395404473730413_5131280576272319091_124376...,2636395404473730413_5131280576272319091_124376...,2636395404473730413_5131280576272319091_124376...,2636395404473730413_5131280576272319091_124376...
478135,5693770660150212848,2636395404473730413,2636395404473730413_5131280576272319091,2636395404473730413_5131280576272319091_124376...,2636395404473730413_5131280576272319091_124376...,2636395404473730413_5131280576272319091_124376...,2636395404473730413_5131280576272319091_124376...
478136,4623253188146764341,2636395404473730413,2636395404473730413_5131280576272319091,2636395404473730413_5131280576272319091_124376...,2636395404473730413_5131280576272319091_124376...,2636395404473730413_5131280576272319091_124376...,2636395404473730413_5131280576272319091_124376...


In [32]:
def encode(data):
    id_features=['shop_id', 'item_id', 'user_id', 'item_brand_id', 'item_city_id', 'user_gender_id','item_property_list', 'predict_category_property',
                      'user_occupation_id', 'context_page_id','top1','top2','top3','top4','top5','top10','query1','query','cate']
    
    for feature in id_features:
        data[feature] = LabelEncoder().fit_transform(data[feature])
    
    return data

In [33]:
encode(train)

,instance_id,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,...,query1,query,cate,top,top1,top2,top3,top4,top5,top10
0,108641074714126964,3760,7908382889764677758;5799347067982556520,2260,447,49,3.0,3.0,4.0,14.0,...,152,9325,9,"(2636395404473730413, 2636395404473730413_5131...",2,20,117,281,450,741
1,5754713551599725161,3760,7908382889764677758;5799347067982556520,2260,447,49,3.0,3.0,4.0,14.0,...,152,10370,9,"(2636395404473730413, 2636395404473730413_5131...",2,20,117,281,450,741
2,842679481291040981,3760,7908382889764677758;5799347067982556520,2260,447,49,3.0,3.0,4.0,14.0,...,152,10333,9,"(2636395404473730413, 2636395404473730413_5131...",2,20,117,281,450,741
3,937088850059189027,3760,7908382889764677758;5799347067982556520,2260,447,49,3.0,3.0,4.0,14.0,...,128,3677,9,"(2636395404473730413, 2636395404473730413_5131...",2,20,117,281,450,741
4,7975697065017708072,3760,7908382889764677758;5799347067982556520,2260,447,49,3.0,3.0,4.0,14.0,...,152,10370,9,"(2636395404473730413, 2636395404473730413_5131...",2,20,117,281,450,741
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478133,5940763769799191887,5531,7908382889764677758;5755694407684602296,5624,583,49,8.0,10.0,13.0,14.0,...,242,5887,8,"(2636395404473730413, 2636395404473730413_5131...",2,20,114,275,430,687
478134,3387284546470665526,5531,7908382889764677758;5755694407684602296,5624,583,49,8.0,10.0,13.0,14.0,...,150,2798,8,"(2636395404473730413, 2636395404473730413_5131...",2,20,114,275,430,687
478135,5693770660150212848,5531,7908382889764677758;5755694407684602296,5624,583,49,8.0,10.0,13.0,14.0,...,150,10179,8,"(2636395404473730413, 2636395404473730413_5131...",2,20,114,275,430,687
478136,4623253188146764341,5531,7908382889764677758;5755694407684602296,5624,583,49,8.0,10.0,13.0,14.0,...,242,2120,8,"(2636395404473730413, 2636395404473730413_5131...",2,20,114,275,430,687


In [36]:
train.to_csv("./origin_concat.csv", index=False)

# base_feature.py

In [38]:
processor = cpu_count() - 2
processor

10

In [ ]:
def query_data_prepare():
    data=pd.read_csv('./origin_concat.csv')
    
    data=data[data.day>=6]
    data = data.sort_values(by=['user_id', 'context_timestamp']).reset_index(drop=True)
    users = pd.DataFrame(list(set(data['user_id'].values)), columns=['user_id'])
    l_data = len(users)
    size = math.ceil(l_data / processor)
    
    for i in range(processor):
        start = size * i
        end = (i + 1) * size if (i + 1) * size < l_data else l_data
        user = users[start:end]
        t_data = pd.merge(data, user, on='user_id').reset_index(drop=True)
        t_data.to_csv('./query_'+str(i)+'.csv',index=False)
        print(len(t_data))

In [40]:
train.day.value_counts()

18.0    78268
21.0    71199
19.0    70931
20.0    68387
22.0    68318
23.0    63614
24.0    57421
Name: day, dtype: int64